In [23]:
import pandas as pd
import numpy as np

In [24]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [25]:
combine = [train_df, test_df]

In [26]:
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [27]:
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

In [28]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\1-14\AppData\Local\Temp\ipykernel_13320\1090031181.py:2: SyntaxWarning: invalid escape sequence '\.'
  dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)


Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [29]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Johkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [30]:
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Jonkheer,0.000000
1,Master,0.575000
2,Miss,0.702703
3,Mr,0.156673
4,Mrs,0.793651
5,Rare,0.363636


In [31]:
title_mapping = {
    'Mr': 1,
    'Miss': 2,
    'Mrs': 3,
    'Master': 4,
    'Rare':5
}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,3.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,3.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1.0


In [32]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 9), (418, 9))

In [33]:
sex_mapping = {
    'female': 1,
    'male': 0
}
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,1.0
1,1,1,1,38.0,1,0,71.2833,C,3.0
2,1,3,1,26.0,0,0,7.9250,S,2.0
3,1,1,1,35.0,1,0,53.1000,S,3.0
4,0,3,0,35.0,0,0,8.0500,S,1.0


In [40]:
con_df = pd.concat([train_df, test_df])

In [41]:
con_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,PassengerId
0,0.0,3,0,22.0,1,0,7.2500,S,1.0,NaN
1,1.0,1,1,38.0,1,0,71.2833,C,3.0,NaN
2,1.0,3,1,26.0,0,0,7.9250,S,2.0,NaN
3,1.0,1,1,35.0,1,0,53.1000,S,3.0,NaN
4,0.0,3,0,35.0,0,0,8.0500,S,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
413,NaN,3,0,NaN,0,0,8.0500,S,1.0,1305.0
414,NaN,1,1,39.0,0,0,108.9000,C,5.0,1306.0
415,NaN,3,0,38.5,0,0,7.2500,S,1.0,1307.0
416,NaN,3,0,NaN,0,0,8.0500,S,1.0,1308.0


In [52]:
cond1 = con_df['Sex'] == 0
male_mode = con_df.loc[cond1, 'Age'].mode()
print(male_mode[0])

cond2 = test_df['Sex'] == 0
test_df.loc[cond2, 'Age'] = test_df.loc[cond2, 'Age'].fillna(male_mode[0])

cond3 = con_df['Sex'] == 1
female_mode = con_df.loc[cond3, 'Age'].mode()
print(female_mode[0])

cond4 = test_df['Sex'] == 1
test_df.loc[cond4, 'Age'] = test_df.loc[cond4, 'Age'].fillna(female_mode[0])

21.0
22.0


In [53]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    int64   
 1   Pclass    891 non-null    int64   
 2   Sex       891 non-null    int64   
 3   Age       714 non-null    float64 
 4   SibSp     891 non-null    int64   
 5   Parch     891 non-null    int64   
 6   Fare      891 non-null    float64 
 7   Embarked  889 non-null    object  
 8   Title     891 non-null    float64 
 9   AgeBand   714 non-null    category
dtypes: category(1), float64(3), int64(5), object(1)
memory usage: 63.9+ KB


In [46]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)

train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,0,3,0,22.0,1,0,7.2500,S,1.0,"(16.336, 32.252]"
1,1,1,1,38.0,1,0,71.2833,C,3.0,"(32.252, 48.168]"
2,1,3,1,26.0,0,0,7.9250,S,2.0,"(16.336, 32.252]"
3,1,1,1,35.0,1,0,53.1000,S,3.0,"(32.252, 48.168]"
4,0,3,0,35.0,0,0,8.0500,S,1.0,"(32.252, 48.168]"
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,S,5.0,"(16.336, 32.252]"
887,1,1,1,19.0,0,0,30.0000,S,2.0,"(16.336, 32.252]"
888,0,3,1,NaN,1,2,23.4500,S,2.0,NaN
889,1,1,0,26.0,0,0,30.0000,C,1.0,"(16.336, 32.252]"


In [49]:
combine = [train_df, test_df]
combine

[     Survived  Pclass  Sex  Age  SibSp  Parch     Fare Embarked  Title  \
 0           0       3    0  1.0      1      0   7.2500        S    1.0   
 1           1       1    1  2.0      1      0  71.2833        C    3.0   
 2           1       3    1  1.0      0      0   7.9250        S    2.0   
 3           1       1    1  2.0      1      0  53.1000        S    3.0   
 4           0       3    0  2.0      0      0   8.0500        S    1.0   
 ..        ...     ...  ...  ...    ...    ...      ...      ...    ...   
 886         0       2    0  1.0      0      0  13.0000        S    5.0   
 887         1       1    1  1.0      0      0  30.0000        S    2.0   
 888         0       3    1  NaN      1      2  23.4500        S    2.0   
 889         1       1    0  1.0      0      0  30.0000        C    1.0   
 890         0       3    0  1.0      0      0   7.7500        Q    1.0   
 
               AgeBand  
 0    (16.336, 32.252]  
 1    (32.252, 48.168]  
 2    (16.336, 32.252] 

In [50]:
for dataset in combine:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age'] = 4

In [51]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,0,3,0,0.0,1,0,7.2500,S,1.0,"(16.336, 32.252]"
1,1,1,1,0.0,1,0,71.2833,C,3.0,"(32.252, 48.168]"
2,1,3,1,0.0,0,0,7.9250,S,2.0,"(16.336, 32.252]"
3,1,1,1,0.0,1,0,53.1000,S,3.0,"(32.252, 48.168]"
4,0,3,0,0.0,0,0,8.0500,S,1.0,"(32.252, 48.168]"
